In [ ]:
# Mounting Drive
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
# Specify YOUR working directory:
main_dir = 'YOUR_MAIN_DIR'

# Setup:

In [ ]:
%%javascript
function ClickConnect(){
console.log("Working");
document.querySelector("colab-toolbar-button#connect").click()
}setInterval(ClickConnect,60000)

<IPython.core.display.Javascript object>

In [ ]:
!nvidia-smi

Tue Mar  1 13:02:01 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install wandb
import wandb
wandb.login()

# Installing Packages

In [ ]:
import os
os.chdir(main_dir)

!pip install './install/torch-1.7.1+cu110-cp37-cp37m-linux_x86_64.whl' --no-deps
!pip install './install/torchvision-0.8.2+cu110-cp37-cp37m-linux_x86_64.whl' --no-deps
!pip install './install/torchaudio-0.7.2-cp37-cp37m-manylinux1_x86_64.whl' --no-deps

!pip install './install/addict-2.4.0-py3-none-any.whl' --no-deps
!pip install './install/yapf-0.31.0-py2.py3-none-any.whl' --no-deps
!pip install './install/terminal-0.4.0-py3-none-any.whl' --no-deps
!pip install './install/terminaltables-3.1.0-py3-none-any.whl' --no-deps
!pip install './install/mmcv_full-1.3.17-cp37-cp37m-manylinux1_x86_64.whl' --no-deps
!pip install './install/pycocotools-2.0.2' --no-deps
!pip install './install/mmpycocotools-12.0.3' --no-deps

%cd ./install/mmdetection-2.18.0
!pip install -e .
import mmdet

!pip install albumentations==0.4.6

os.chdir(main_dir)

Processing ./install/torch-1.7.1+cu110-cp37-cp37m-linux_x86_64.whl
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0+cu111
    Uninstalling torch-1.10.0+cu111:
      Successfully uninstalled torch-1.10.0+cu111
Processing ./install/torchvision-0.8.2+cu110-cp37-cp37m-linux_x86_64.whl
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.11.1+cu111
    Uninstalling torchvision-0.11.1+cu111:
      Successfully uninstalled torchvision-0.11.1+cu111
Processing ./install/torchaudio-0.7.2-cp37-cp37m-manylinux1_x86_64.whl
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 0.10.0+cu111
    Uninstalling torchaudio-0.10.0+cu111:
      Successfully uninstalled torchaudio-0.10.0+cu111
Processing ./install/addict-2.4.0-py3-none-any.whl
Processing ./install/yapf-0.31.0-py2.py3-none-any.whl
Processing ./install/terminal-0.4.0-py3-none-any.whl
Processing ./install/terminaltables-3.1.0-py3-none-any.whl
Processing ./install/

In [ ]:
import torch
print(torch.__version__)
print("mmcv_full:")
!pip show mmcv_full |grep Version
print("mmdet:")
!pip show mmdet |grep Version

1.7.1+cu110
mmcv_full:
Version: 1.3.17
mmdet:
Version: 2.18.0


# Importations

In [ ]:
# Importations
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os, json, cv2, random
import seaborn as sns
import numpy.random
import random

import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torch.nn.functional as F
import sklearn
import torchvision
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import PIL
from PIL import Image, ImageEnhance
import albumentations as A
import mmdet
import mmcv
from mmcv import Config
import albumentations 
from albumentations.pytorch import ToTensorV2

import glob
from pathlib import Path
import pycocotools
from pycocotools import mask
import re

from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.apis import inference_detector, init_detector, show_result_pyplot, set_random_seed


# Model Config

In [ ]:
config_path = main_dir + 'models/det/det2_cascade_rcnn_resnest/configs/config_det2_cascade_rcnn_resnest.py'

In [ ]:
cfg = Config.fromfile(config_path)
meta = dict()
print(cfg.pretty_text)

# Model Training

In [ ]:
datasets = [build_dataset(cfg.data.train)]
model = build_detector(cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))
model.CLASSES = datasets[0].CLASSES

In [ ]:
mmcv.mkdir_or_exist(os.path.abspath(cfg.work_dir))
train_detector(model, datasets, cfg, distributed=False, validate=True, meta=meta)